# Assignment 2
### Gauranga Das a1910652

## A. Tasks as specified for your team structure

**One headings for each task.**

In [1]:
# INSTALL REQUIRED LIBRARIES
!pip install torch
!pip install torchtext==0.6.0
!python -m spacy download en_core_web_sm
!pip install fastcoref
!pip install transformers
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [2]:
import pandas as pd
import torch
import random
from torch import nn
from fastcoref import spacy_component
import spacy
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils import clip_grad_norm_
import torchtext
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import unicodedata
import re
from torchtext.data import Field, BucketIterator,TabularDataset
from sklearn.model_selection import train_test_split
from transformers import MobileBertModel, MobileBertTokenizer, AutoTokenizer, AutoModel
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize
import json
from sentence_transformers import SentenceTransformer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# Set seed for reproducibility
torch.manual_seed(10)
np.random.seed(10)
random.seed(10)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [4]:
# Mount Google Colab if needed
# from google.colab import drive
# drive.mount('/content/drive')

**The variable in the below cell decides whether to train the model or not. Set the the variable to `False` to skip the data processing and fine-tuning process and simply test the model. In that case, the model weights will be downloaded from my Github repository. The model weights were obtained after data preprocessing and fine-tuning the model as shown below. If the variable is set to `True`, data preprocessing and fine-tuning will take place which can take a lot of time.**

In [5]:
TRAIN_MODEL = True

**The entire process of data processing and fine-tuning the model takes a lot of time even on a GPU I have uploaded the model weights and notebook on Github. The cell below downloads the model weights that I have obtained after fine-tuning on the SQuAD v2.0 dataset below.**

In [6]:
if not TRAIN_MODEL:
    !wget https://github.com/dasgauranga2/Applied-NLP-Assignment-2/raw/main/mobilebert_squad.pth

**GITHUB**

[Github Link](https://github.com/dasgauranga2/Applied-NLP-Assignment-2/tree/main)

**READ AND PREPROCESS THE DATA**

In [7]:
def preprocess(text):
    '''
    Apply preprocessing to a piece of text
    text is of type string

    REFERENCES:
    - Fredtantini (2014)
    - MiniQuark (2009)
    '''

    # Convert text to lowercase
    text = text.lower()

    # Remove unnecessary whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Normalization of text
    normalized = unicodedata.normalize('NFKD', text)
    text = normalized.encode('ascii', 'ignore').decode('ascii')

    return text

In [8]:
# REFERENCES:
# - Shamporov (2024)
# MobileBERT Tokenizer
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/847 [00:00<?, ?B/s]

In [9]:
if TRAIN_MODEL:
    # REFERENCES:
    # - Rajpurkar, Jia & Liang (2018)
    # Download SQuADv2.0 dataset
    !wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json

--2025-01-13 01:33:15--  https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json
Resolving rajpurkar.github.io (rajpurkar.github.io)... 185.199.108.153, 185.199.110.153, 185.199.111.153, ...
Connecting to rajpurkar.github.io (rajpurkar.github.io)|185.199.108.153|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 42123633 (40M) [application/json]
Saving to: ‘train-v2.0.json’

train-v2.0.json     100%[===================>]  40.17M  23.8MB/s    in 1.7s    

2025-01-13 01:33:18 (23.8 MB/s) - ‘train-v2.0.json’ saved [42123633/42123633]



In [10]:
if TRAIN_MODEL:
    # REFERENCES:
    # - Rajpurkar, Jia & Liang (2018)
    # Read the SQuADv2.0 json data and load it
    with open('train-v2.0.json') as file:
        jstr = file.read()
        squad_data = json.loads(jstr)

In [11]:
def find_answer_indices(context_tokens, answer_tokens):
    '''
    Given the context tokens and answer tokens find the start and end indices of the answer
    context_tokens: list of context tokens
    answer_tokens: list of answer tokens
    '''

    for i in range(len(context_tokens) - len(answer_tokens) + 1):
        if context_tokens[i:i+len(answer_tokens)] == answer_tokens:
            return i, i + len(answer_tokens) - 1
    return -1, -1

In [12]:
if TRAIN_MODEL:

    final_data = []

    # Process the SQuADv2.0 dataset by looping through each paragraph and corresponding questions and answers
    # Concatenate each paragraph and question seperated by the [SEP] token
    # Then find the answer indices for each
    # Finally, store each paragraph-question pair and corresponding answer indices in a list
    for data in squad_data['data']:
        for para in data['paragraphs']:
            context = preprocess(para['context'])
            context_tokens = tokenizer.tokenize(context)
            if len(context_tokens)<240:
                for question in para['qas']:
                    ques = preprocess(question['question'])
                    if not question['is_impossible']:
                        answer = preprocess(question['answers'][0]['text'])
                        start,end = find_answer_indices(context_tokens,tokenizer.tokenize(answer))
                        if start != -1 and end != -1:
                            final_data.append([context + ' [SEP] ' + ques,start+1,end+1])
                    else:
                        final_data.append([context + ' [SEP] ' + ques,0,0])


    random.shuffle(final_data)

In [13]:
if TRAIN_MODEL:
    print(len(final_data))

117606


In [14]:
if TRAIN_MODEL:
    # Create a Pandas dataset
    squad_df = pd.DataFrame(final_data,columns = ['Input','Start','End'])

    # Length of the training set
    train_len = int(len(final_data)*0.8)

    # Split the dataset into training set and validation set
    squad_df[:train_len].to_csv('squad_train.csv',index=False)
    squad_df[train_len:].to_csv('squad_valid.csv',index=False)

    squad_df.head(10)

In [15]:
# Special tokens of MobileBERT

# Start of sequence token
init_token = tokenizer.cls_token
# End of sequence and sequence seperator token
eos_token = tokenizer.sep_token
# Padding token
pad_token = tokenizer.pad_token
# Token for unknown subwords
unk_token = tokenizer.unk_token

In [16]:
# Get the indices for each special token
init_token_idx = tokenizer.convert_tokens_to_ids(init_token)
eos_token_idx = tokenizer.convert_tokens_to_ids(eos_token)
pad_token_idx = tokenizer.convert_tokens_to_ids(pad_token)
unk_token_idx = tokenizer.convert_tokens_to_ids(unk_token)

In [17]:
# Maximum input length
max_input_length = tokenizer.model_max_length

In [18]:
def tokenize_and_cut(txt):
    '''
    Tokenize text and truncate it to the maximum length
    '''

    tokens = tokenizer.tokenize(txt)
    tokens = tokens[:max_input_length-2]
    return tokens

In [19]:
if TRAIN_MODEL:

    # Torchtext fields specify how data will be processed
    # Each field specifies how each column of dataset will be processed

    # TEXT field is used to process the input text
    TEXT = Field(batch_first = True,
                use_vocab = False, # 'use_vocab' is set to False because tokenizer already has it's own vocabulary
                tokenize = tokenize_and_cut, # Tokenizer function
                preprocessing = tokenizer.convert_tokens_to_ids, # Function to convert tokens to numerical indices
                init_token = init_token_idx,
                eos_token = eos_token_idx,
                pad_token = pad_token_idx,
                unk_token = unk_token_idx)

    # START field is used to process the answer start indices
    START = Field(sequential=False, use_vocab=False, dtype=torch.long)
    # END field is used to process the answer end indices
    END = Field(sequential=False, use_vocab=False, dtype=torch.long)

In [20]:
if TRAIN_MODEL:
    # Names for each field
    fields = [('text', TEXT),('start', START),('end',END)]

In [21]:
if TRAIN_MODEL:
    # Load the Pandas dataset that was saved
    train_data,valid_data = TabularDataset.splits(path = '',
                                    train = 'squad_train.csv',
                                    test = 'squad_valid.csv',
                                    format = 'csv',
                                    fields = fields,
                                    skip_header = True)

In [22]:
if TRAIN_MODEL:
    # Load a training example
    print(vars(train_data.examples[5000]))

{'text': [1996, 2240, 1997, 17189, 2015, 1997, 1996, 2442, 11475, 19214, 2072, 20474, 7486, 1006, 2036, 2124, 2004, 1996, 8945, 13492, 2015, 1013, 4830, 3702, 2072, 8945, 13492, 1007, 2506, 2039, 2000, 9779, 14503, 7839, 2442, 11475, 1012, 2044, 2010, 2331, 1010, 2027, 2903, 2037, 7398, 17189, 13843, 3148, 2497, 11113, 2072, 2632, 1011, 1053, 21369, 2213, 2253, 2046, 1037, 4830, 13088, 1011, 1041, 1011, 2938, 2099, 1006, 2558, 1997, 19819, 3672, 1007, 2008, 4247, 2000, 2023, 2154, 1012, 1999, 1996, 6438, 1997, 2019, 17189, 2027, 2024, 2419, 2011, 1037, 18765, 1011, 2632, 1011, 14163, 19646, 20784, 1006, 7619, 8696, 1007, 2040, 9020, 1996, 3821, 1997, 1996, 17189, 1011, 1999, 1011, 19819, 3672, 2127, 2128, 1011, 14053, 1997, 1996, 17189, 2013, 19819, 3672, 1012, 4830, 3702, 2072, 8945, 13492, 1005, 1055, 2556, 5187, 4103, 4830, 1005, 1045, 2632, 1011, 14163, 19646, 20784, 2003, 2010, 27692, 19740, 2532, 14163, 7011, 25062, 2140, 18952, 11263, 18277, 1006, 10722, 2015, 1007, 2040, 4594, 

In [23]:
if TRAIN_MODEL:
    # Convert the numerical indices to tokens
    tokens = tokenizer.convert_ids_to_tokens(vars(train_data.examples[5000])['text'])

    print(tokens)

['the', 'line', 'of', 'imam', '##s', 'of', 'the', 'must', '##ali', 'ismail', '##i', 'shia', 'muslims', '(', 'also', 'known', 'as', 'the', 'bo', '##hra', '##s', '/', 'da', '##wood', '##i', 'bo', '##hra', ')', 'continued', 'up', 'to', 'aa', '##mir', 'ibn', 'must', '##ali', '.', 'after', 'his', 'death', ',', 'they', 'believe', 'their', '21st', 'imam', 'tai', '##ya', '##b', 'ab', '##i', 'al', '-', 'q', '##asi', '##m', 'went', 'into', 'a', 'da', '##wr', '-', 'e', '-', 'sat', '##r', '(', 'period', 'of', 'conceal', '##ment', ')', 'that', 'continues', 'to', 'this', 'day', '.', 'in', 'the', 'absence', 'of', 'an', 'imam', 'they', 'are', 'led', 'by', 'a', 'dai', '-', 'al', '-', 'mu', '##tl', '##aq', '(', 'absolute', 'missionary', ')', 'who', 'manages', 'the', 'affairs', 'of', 'the', 'imam', '-', 'in', '-', 'conceal', '##ment', 'until', 're', '-', 'emergence', 'of', 'the', 'imam', 'from', 'conceal', '##ment', '.', 'da', '##wood', '##i', 'bo', '##hra', "'", 's', 'present', '53', '##rd', 'da', "'", 

In [24]:
# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

cuda


In [25]:
if TRAIN_MODEL:
    # Batch size
    BATCH_SIZE = 32

    # Create data iterators using the dataset that will load a batch of data
    train_iterator,valid_iterator = BucketIterator.splits(
        (train_data,valid_data),
        batch_size = BATCH_SIZE,
        sort_key = lambda x: len(x.text),
        sort_within_batch=True,
        device = device)

**CHOOSE MODEL AND HYPERPARAMETERS**

In [26]:
class BERTQA(nn.Module):
    '''
    Neural Network model for predicting the answer indices
    '''

    def __init__(self):
        super().__init__()

        # REFERENCES:
        # - Shamporov (2024)
        # - Sun et al. (2020)
        # Get the MobileBERT model from HuggingFace
        self.bert = MobileBertModel.from_pretrained('google/mobilebert-uncased')

        # MobileBERT hidden dimension
        bert_hidden_dim = self.bert.config.to_dict()['hidden_size']

        # Fully-connected layer to predict the start index of answer
        self.start = nn.Linear(bert_hidden_dim, 1)
        # Fully-connected layer to predict the end index of answer
        self.end = nn.Linear(bert_hidden_dim, 1)

    def forward(self, text):
        # text : [batch size, sequence length]

        # generate attention mask for mobilebert inputs
        attention_mask = (text != pad_token_idx).long()
        embedded = self.bert(text,attention_mask=attention_mask)[0]
        # embedded = [batch size, sequence length, mobilebert hidden dimension]

        start_positions = self.start(embedded).squeeze()
        end_positions = self.end(embedded).squeeze()

        # start_positions, end_positions : [batch size, sequence length]
        return start_positions, end_positions

In [27]:
# Initialize the model
model = BERTQA().to(device)

# Path of model weights if they exist
model_path = 'mobilebert_squad.pth'

if not TRAIN_MODEL:
    # Load the model weights
    try:
        model.load_state_dict(torch.load(model_path,map_location=device))
        print("Model weights loaded")
    except:
        print("No model weights detected")

pytorch_model.bin:   0%|          | 0.00/147M [00:00<?, ?B/s]

In [28]:
if TRAIN_MODEL:
    # Optimizer
    optimizer = torch.optim.AdamW(model.parameters(),lr=0.00004)

    # Loss function
    criterion = nn.CrossEntropyLoss().to(device)

In [29]:
def calculate_f1(actual_answer, pred_answer):
    '''
    Calculate the F1 score for a single predicted answer and actual/gold answer
    REFERENCES:
    - Kim (2018)
    '''

    actual_tokens = tokenizer.tokenize(actual_answer)
    pred_tokens = tokenizer.tokenize(pred_answer)
    common_tokens = Counter(actual_tokens) & Counter(pred_tokens)

    num_same = sum(common_tokens.values())
    if num_same == 0:
        return 0

    precision = num_same / len(pred_tokens)
    recall = num_same / len(actual_tokens)
    if precision + recall == 0:
        return 0
    f1 = (2 * precision * recall) / (precision + recall)

    return f1

def calculate_average_f1(actual_answers, pred_answers):
    '''
    Calculate the F1 score for a list of predicted answers and actual/gold answers
    '''

    f1_scores = []
    for actual, pred in zip(actual_answers, pred_answers):
        f1_scores.append(calculate_f1(actual, pred))

    # if len(f1_scores)==0: # prevent division by zero
    #     return 0
    average_f1 = sum(f1_scores) / len(f1_scores)
    return average_f1

In [30]:
def train(model, iterator, optimizer, criterion):
    '''
    Training the neural network for an epoch
    model : Neural network model defined above
    iterator : data iterator
    optimizer : Optimizer defined above
    criterion: Loss function
    '''

    epoch_loss = 0

    # Set model to training mode so that it can be trained
    model.train()

    # Iterate over each batch of data
    for batch in iterator:

        # Clear the gradients
        optimizer.zero_grad()

        # Pass the input to the model
        start_preds,end_preds = model(batch.text)

        # Calculate the loss
        loss = criterion(start_preds, batch.start) + criterion(end_preds, batch.end)

        # Calculate the gradients of the loss function
        loss.backward()

        # REFERENCES:
        # - Metsai (2016)
        # Clip the gradient values to prevent them from becoming too large
        clip_grad_norm_(model.parameters(), 1)

        # Update the model parameters
        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss/len(iterator)

In [31]:
def evaluate(model, iterator, criterion):
    '''
    Evaluating the neural network for an epoch on the development dataset
    model : Neural network model defined above
    iterator : data iterator
    criterion: Loss function
    '''

    epoch_loss = 0
    all_f1 = []

    # Set model to evaluation mode
    model.eval()

    # This line disables gradient calculation
    with torch.no_grad():

        # Iterate over each batch of data
        for batch in iterator:

            # Pass the input to the model
            start_preds,end_preds = model(batch.text)

            # Calculate the loss
            loss = criterion(start_preds, batch.start) + criterion(end_preds, batch.end)

            epoch_loss += loss.item()

            # REFERENCES:
            # - Shamporov (2024)
            # The code below calculates the F1 score for the batch of data
            pred_start = torch.argmax(start_preds, dim=1)
            pred_end = torch.argmax(end_preds, dim=1)

            for i in range(batch.text.size(0)):
                actual_start = batch.start[i].item()
                actual_end = batch.end[i].item()
                actual_tokens = tokenizer.convert_ids_to_tokens(batch.text[i][actual_start:actual_end+1])
                actual_answer = tokenizer.convert_tokens_to_string(actual_tokens)

                pred_s = pred_start[i].item()
                pred_e = pred_end[i].item()
                pred_tokens = tokenizer.convert_ids_to_tokens(batch.text[i][pred_s:pred_e+1])
                pred_answer = tokenizer.convert_tokens_to_string(pred_tokens)

                f1 = calculate_f1(actual_answer, pred_answer)
                all_f1.append(f1)

    return epoch_loss/len(iterator) , sum(all_f1)/len(all_f1)

In [32]:
if TRAIN_MODEL:

    # Number of epochs
    EPOCHS = 4

    for epoch in range(EPOCHS):

        # Train the model on the training dataset
        train_loss = train(model, train_iterator, optimizer, criterion)
        # Save the model weights if needed
        torch.save(model.state_dict(), model_path)
        # Evaluate the model on the development dataset
        valid_loss,valid_f1 = evaluate(model, valid_iterator, criterion)

        print(f'Epoch: {epoch+1:02}')
        print(f'\tTrain Loss: {train_loss:.2f}')
        print(f'\tValid Loss: {valid_loss:.2f}\tValid F1: {valid_f1*100:.2f}%\n')

Epoch: 01
	Train Loss: 26106.10
	Valid Loss: 2.04	Valid F1: 68.93%

Epoch: 02
	Train Loss: 1.87
	Valid Loss: 1.81	Valid F1: 73.98%

Epoch: 03
	Train Loss: 1.49
	Valid Loss: 1.81	Valid F1: 75.32%

Epoch: 04
	Train Loss: 1.21
	Valid Loss: 1.81	Valid F1: 76.14%



**TESTING**

In [33]:
def predict_answer(model, tokenizer, context, query):
    '''
    Given the model, tokenizer, context and question
    return the answer predicted by the model
    '''

    input_tokens = tokenizer.tokenize(context + ' [SEP] ' + query)
    indexed = [init_token_idx] + tokenizer.convert_tokens_to_ids(input_tokens) + [eos_token_idx]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    start_preds,end_preds = model(tensor)
    start_idx = start_preds.squeeze().argmax().item()
    end_idx = end_preds.squeeze().argmax().item()

    # If the model predicts the start index or end index to be 0
    # That means that the model thinks that the answer does not exists in the context
    if start_idx==0 or end_idx==0:
        return 'NO ANSWER'
    else:
        return tokenizer.convert_tokens_to_string(input_tokens[start_idx-1:end_idx])

In [34]:
def find_most_relevant_sent(sentences, query):
    '''
    Find the most relevant sentence using Sentence-BERT
    sentences : list of sentences
    query : question given by the user
    '''

    # REFERENCES:
    # - Aarsen (2022)
    # Initialize the Sentence-BERT model
    model = SentenceTransformer('all-MiniLM-L6-v2')

    # Get embeddings for each sentence
    sentence_embeddings = model.encode(sentences)
    # Get embeddings for the query
    query_embedding = model.encode([query])

    # REFERENCES:
    # - Natchiappan (2019)
    # Calculate cosine similarities between the query and all sentences
    similarities = np.squeeze(cosine_similarity(query_embedding, sentence_embeddings))

    # Find the index of the most relevant sentence
    most_relevant_index = np.argmax(similarities)

    # Find the most relevant sentence
    most_relevant_sentence = sentences[most_relevant_index]

    # Get the confidence score
    confidence_score = similarities[most_relevant_index]

    return most_relevant_sentence, confidence_score

In [35]:
# News dataset path
news_path = 'news_dataset.csv'

# Load the News dataset
news_df = pd.read_csv(news_path,encoding='ISO-8859-1')

news_df.head()

,id,author,date,year,month,topic,article
0,17307,Marlise Simons,1/01/2017,2017,1,architecture,PARIS ? When the Islamic State was about to...
1,17292,Andy Newman,31/12/2016,2016,12,art,Angels are everywhere in the Mu?iz family?s ap...
2,17298,Emma G. Fitzsimmons,2/01/2017,2017,1,business,Finally. The Second Avenue subway opened in Ne...
3,17311,Carl Hulse,3/01/2017,2017,1,business,WASHINGTON ? It?s or time for Republica...
4,17339,Jim Rutenberg,5/01/2017,2017,1,business,"For Megyn Kelly, the shift from Fox News to NB..."


In [36]:
# Get the article number from the user
article_number = input('Enter article number : ')

# Check if a valid article number is entered by the user
if article_number.isdigit() and int(article_number) > 0:

    article = news_df.loc[news_df['id']==int(article_number)]
    if article.empty:
        print('INVALID ARTICLE NUMBER')
    else:
        article = preprocess(article.iloc[0,6])
        print(article)
else:
    print('INVALID INPUT')

Enter article number : 17574
seoul, south korea ? a special prosecutor investigating the corruption scandal that led to president park ?s impeachment summoned the de facto head of samsung for questioning on wednesday, calling him a bribery suspect. the de facto leader, jay y. lee, the vice chairman of samsung, will be questioned on thursday, according to the special prosecutor?s office, which recommended that he also be investigated on suspicion of perjury. mr. lee effectively runs samsung, south korea?s largest conglomerate he is the son of its chairman, lee who has been incapacitated with health problems. he is expected to be asked whether donations that samsung made to two foundations controlled by choi a longtime friend of the president, amounted to bribes, and what role, if any, he played in the decision to give the money. investigators at the special prosecutor?s office have questioned other senior samsung executives as suspects about the bribery accusations. neither samsung nor 

In [39]:
# REFERENCES:
# - Slider (2016)
# Split the article into a list of sentences
nltk.download('punkt_tab')
sentences = sent_tokenize(article)

In [40]:
# Get the questions and corresponding answers from the user

questions = []
answers = []
user_question = ''
user_answer = ''

while True:
    user_question = input("Enter the question (Type 'quit' to stop): ")
    # Check if the user has typed 'quit'
    if user_question == 'quit':
        break
    user_answer = input('Enter the answer: ')
    questions.append(preprocess(user_question))
    answers.append(preprocess(user_answer))

print(questions)
print(answers)

Enter the question (Type 'quit' to stop): who is the vice chairman of samsung
Enter the answer: jay y. lee
Enter the question (Type 'quit' to stop): who is the leader of samsung
Enter the answer: jay y. lee
Enter the question (Type 'quit' to stop): who was impeached
Enter the answer: president park
Enter the question (Type 'quit' to stop): on what day was the vice chairman questioned
Enter the answer: thursday
Enter the question (Type 'quit' to stop): when was jay y. lee summoned
Enter the answer: wednesday
Enter the question (Type 'quit' to stop): who controlled the foundations that received the donations
Enter the answer: choi
Enter the question (Type 'quit' to stop): who voted to impeach president park
Enter the answer: the national assembly
Enter the question (Type 'quit' to stop): who can end president park's presidency
Enter the answer: the constitutional court
Enter the question (Type 'quit' to stop): who is the chairman of the pension fund
Enter the answer: moon
Enter the quest

In [41]:
pred_answers = []

for i in range(len(questions)):

    # For each question, find the most relevant sentence
    most_relevant_sentence,cs = find_most_relevant_sent(sentences,questions[i])
    # Use the most relevant sentence and question to predict the answer
    pred_answer = predict_answer(model, tokenizer, most_relevant_sentence, questions[i])

    print(f"QUESTION: {questions[i]}")
    print(f"ACTUAL ANSWER: {answers[i]}")
    print(f"MOST RELEVANT SENTENCE: {most_relevant_sentence}")
    print(f"CONFIDENCE SCORE: {cs*100:.2f}%")
    print(f"PREDICTED ANSWER: {pred_answer}\n")

    pred_answers.append(pred_answer)

# Calculate the F1 score using the predicted answers and actual answers
f1 = calculate_average_f1(answers, pred_answers)

print(f"F1 SCORE : {f1*100:.2f}%")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

QUESTION: who is the vice chairman of samsung
ACTUAL ANSWER: jay y. lee
MOST RELEVANT SENTENCE: the de facto leader, jay y. lee, the vice chairman of samsung, will be questioned on thursday, according to the special prosecutor?s office, which recommended that he also be investigated on suspicion of perjury.
CONFIDENCE SCORE: 62.71%
PREDICTED ANSWER: jay y . lee

QUESTION: who is the leader of samsung
ACTUAL ANSWER: jay y. lee
MOST RELEVANT SENTENCE: mr. lee effectively runs samsung, south korea?s largest conglomerate he is the son of its chairman, lee who has been incapacitated with health problems.
CONFIDENCE SCORE: 67.50%
PREDICTED ANSWER: mr . lee

QUESTION: who was impeached
ACTUAL ANSWER: president park
MOST RELEVANT SENTENCE: in its impeachment bill, the national assembly asserted that the donations were bribes, made with the expectation of political favors from the president.
CONFIDENCE SCORE: 45.37%
PREDICTED ANSWER: NO ANSWER

QUESTION: on what day was the vice chairman questi

**APPLYING COREFERENCE RESOLUTION UTILITY**

In [42]:
# REFERENCES:
# - Otmazgin (2023)
nlp = spacy.load("en_core_web_sm")
# Add fastcoref to spacy's pipeline
nlp.add_pipe("fastcoref",
             config={'model_architecture': 'LingMessCoref', 'model_path': 'biu-nlp/lingmess-coref', 'device': 'cpu'})
doc = nlp(article,component_cfg={"fastcoref": {'resolve_text': True}})
# Apply coreference resolution to the article
article = doc._.resolved_text

print(article)

config.json:   0%|          | 0.00/1.22k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/361 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36G [00:00<?, ?B/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

seoul, south korea ? a special prosecutor investigating the corruption scandal that led to president park ?s impeachment the announcement on wednesday the de facto head of samsung for questioning on wednesday, calling the de facto head of samsung a bribery suspect. the de facto leader, jay y. lee, the vice chairman of samsung, will be questioned on thursday, according to the special prosecutor?s office, which recommended that the de facto head of samsung also be investigated on suspicion of perjury. the de facto head of samsung effectively runs samsung, south korea?s largest conglomerate the de facto head of samsung is the son of samsung's chairman, lee who has been incapacitated with health problems. the de facto head of samsung is expected to be asked whether donations that samsung made to two foundations controlled by choi a longtime friend of president park ?s, amounted to bribes, and what role, if any, the de facto head of samsung played in the decision to give donations that sams

In [43]:
# REFERENCES:
# - Slider (2016)
# Split the article into a list of sentences
sentences = sent_tokenize(article)

In [44]:
pred_answers = []

for i in range(len(questions)):

    # For each question, find the most relevant sentence
    most_relevant_sentence,cs = find_most_relevant_sent(sentences,questions[i])
    # Use the most relevant sentence and question to predict the answer
    pred_answer = predict_answer(model, tokenizer, most_relevant_sentence, questions[i])

    print(f"QUESTION: {questions[i]}")
    print(f"ACTUAL ANSWER: {answers[i]}")
    print(f"MOST RELEVANT SENTENCE: {most_relevant_sentence}")
    print(f"CONFIDENCE SCORE: {cs*100:.2f}%")
    print(f"PREDICTED ANSWER: {pred_answer}\n")

    pred_answers.append(pred_answer)

# Calculate the F1 score using the predicted answers and actual answers
f1 = calculate_average_f1(answers, pred_answers)

print(f"F1 SCORE : {f1*100:.2f}%")

QUESTION: who is the vice chairman of samsung
ACTUAL ANSWER: jay y. lee
MOST RELEVANT SENTENCE: the de facto leader, jay y. lee, the vice chairman of samsung, will be questioned on thursday, according to the special prosecutor?s office, which recommended that the de facto head of samsung also be investigated on suspicion of perjury.
CONFIDENCE SCORE: 65.27%
PREDICTED ANSWER: jay y . lee

QUESTION: who is the leader of samsung
ACTUAL ANSWER: jay y. lee
MOST RELEVANT SENTENCE: the de facto head of samsung effectively runs samsung, south korea?s largest conglomerate the de facto head of samsung is the son of samsung's chairman, lee who has been incapacitated with health problems.
CONFIDENCE SCORE: 72.76%
PREDICTED ANSWER: lee

QUESTION: who was impeached
ACTUAL ANSWER: president park
MOST RELEVANT SENTENCE: the special prosecutor?s office, which recommended that he also be investigated on suspicion of perjury asked national assembly to file a perjury complaint against the de facto head of

## B. References
Aarsen, T 2022, all-MiniLM-L6-v2, HuggingFace, https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2
<br><br>
Devlin, J, Chang, M-W, Lee, K & Toutanova, K 2019, ‘BERT: Pre-training of Deep Bidirectional Transformers for Language Understanding’, Proceedings of NAACL-HLT 2019, pp. 4171-4186, https://aclanthology.org/N19-1423.pdf
<br><br>
Fredtantini 2014, python - remove whitespaces that are not needed [duplicate], StackOverflow, https://stackoverflow.com/questions/27618462/python-remove-whitespaces-that-are-not-needed
<br><br>
Gugger, S, Zucker, A, Jik, L 2024, transformers/src/transformers/models/mobilebert/tokenization_mobilebert.py, Github, https://github.com/huggingface/transformers/blob/main/src/transformers/models/mobilebert/tokenization_mobilebert.py
<br><br>
Kim, T, 2018, BiDAF-pytorch/evaluate.py, Github, https://github.com/galsang/BiDAF-pytorch/blob/master/evaluate.py
<br><br>
Metsai, A 2016, Pytorch: test loss becoming nan after some iteration, StackOverflow, https://stackoverflow.com/questions/66648432/pytorch-test-loss-becoming-nan-after-some-iteration
<br><br>
MiniQuark 2009, What is the best way to remove accents (normalize) in a Python unicode string?, StackOverflow, https://stackoverflow.com/questions/517923/what-is-the-best-way-to-remove-accents-normalize-in-a-python-unicode-string
<br><br>
Natchiappan, V 2019, Using sklearn how do I calculate the tf-idf cosine similarity between documents and a query?, StackOverflow, https://stackoverflow.com/questions/55677314/using-sklearn-how-do-i-calculate-the-tf-idf-cosine-similarity-between-documents
<br><br>
Otmazgin, S 2023, fastcoref, Github, https://github.com/shon-otmazgin/fastcoref
<br><br>
Rajpurkar, P, Jia, R & Liang, P 2018, SQuAD 2.0(Stanford Question Answering Dataset), https://rajpurkar.github.io/SQuAD-explorer/
<br><br>
Reimers, N, Gurevych, I 2019, ‘Sentence-BERT: Sentence Embeddings using Siamese BERT-Networks’, Proceedings of the 2019 Conference on Empirical Methods in Natural Language Processing, pp. 3982-3992, https://aclanthology.org/D19-1410.pdf
<br><br>
Shamporov, V 2024, MobileBERT, HuggingFace, https://huggingface.co/docs/transformers/model_doc/mobilebert
<br><br>
Slider 2016, Tokenize a paragraph into sentence and then into words in NLTK, StackOverflow, https://stackoverflow.com/questions/37605710/tokenize-a-paragraph-into-sentence-and-then-into-words-in-nltk
<br><br>
Sun, Z, Yu, H, Song, X, Liu, R, Yang, Y & Zhou, D 2020, ‘MobileBERT: a Compact Task-Agnostic BERT for Resource-Limited Devices’, Proceedings of the 58th Annual Meeting of the Association for Computational Linguistics, pp. 2158-2170, https://aclanthology.org/2020.acl-main.195.pdf
<br><br>
Trevett, B 2023, pytorch-sentiment-analysis, Github, https://github.com/bentrevett/pytorch-sentiment-analysis

## C. Appendix
**Chapter A MobileBERT**<br>
The architecture of the MobileBERT (Sun et al. 2020) model is very similar to the original BERT (Devlin et al. 2019) model but it is much smaller. Its depth(number of Transformer encoder layers) is the same as the original BERT, but each layer has a smaller number of parameters. One key difference is that each Transformer encoder layer of MobileBERT has a linear/fully-connected layer at the beginning and end to adjust its input and output dimensions. Another difference is that each layer of MobileBERT has multiple feed-forward networks to ensure the ratio of parameters in the Multi-Head Self-Attention layer and the Feed-Forward networks layers is the same as the original BERT. A technique called knowledge distillation is used where a larger model called the teacher network is trained first which is then used to train the MobileBERT model. This teacher network has the same architecture as the MobileBERT but it has more parameters(comparable to the original BERT model). Before training MobileBERT, the teacher network was trained using the same objectives as the original BERT model. For training MobileBERT, the following three objectives were used:-
<br>**1.  Feature Map Transfer**<br>
The output feature maps of the teacher model and the MobileBERT model at each layer are used to calculate the mean squared error to transfer layer-wise knowledge from the teacher to MobileBERT
<br>**2.  Attention Transfer**<br>
The per-head self-attention distributions of MobileBERT and the teacher model is used to calculate the KL-divergence loss to help the MobileBERT to learn to properly attend to the input sequence like the teacher model
<br>**3.  Pre-training Distillation**<br>
This combines the masked language modeling loss and next sentence prediction loss which is similar to the original BERT model.<br>
Using the three objectives, different combinations can be used for training. The paper discusses three different strategies however, here only the best performing one will be discussed. Progressive Knowledge Transfer (PKT) works by progressively training each Transformer encoder layer separately. First, the lowest layer is trained using the Feature Map Transfer and Attention Transfer objectives and then progressively each of the layers is trained using the same objectives until the final layer is trained. Finally, the Pre-training distillation objective is used to train the MobileBERT model. For training, the masked language modeling and next sentence prediction was used. The exact implementation details are in the paper.

**Chapter B Sentence-BERT**<br>
The Sentence-BERT (Reimers & Gurevych 2019) model is a pre-trained model to generate sentence embeddings that can be used to compare sentences using cosine similarity. Sentence-BERT uses the same architecture as the original BERT (Devlin et al. 2019) model. To get sentence embedding, the input tokens of a sentence are passed through the BERT model and then the mean of all the token output vectors is calculated to derive the sentence embedding vector for the input token sequence. The model is trained using two objective functions:-
<br>**1.  Regression Objective Function**<br>
Two input sentences are passed to the Sentence-BERT model separately and the sentence embedding for each of them is obtained. Then, the cosine similarity between them is calculated and the mean-squared error loss is used as the objective function.
<br>**2.  Classification Objective Function**<br>
Two input sentences are passed to the Sentence-BERT model separately and the sentence embedding for each of them is obtained. The sentence embeddings are concatenated along with their element-wise difference and then the result is multiplied by a trainable weight to produce a vector whose length is the no. of classes which is then used to train the model using cross-entropy loss.
Furthermore, the model is fine-tuned on two tasks:-
<br>**1.  Natural Language Inference (NLI) Task**<br>
The model is given a premise sentence and it has to predict whether a second sentence is True, False or undetermined. This trains the model to understand the language semantics and basic reasoning skills. For this task the classification objective function is used.
<br>**2.  Semantic Textual Similarity (STS) Task**<br>
The model is given two sentences and it has to predict a degree of similarity between those sentences by giving a score from 0 to 5. This helps the model to determine whether two sentences are similar or not. For this task the regression objective function is used.
The resulting model after training on these tasks is called Sentence-BERT. Given two sentences it can compute the similarity between them using cosine similarity.
